In [1]:
CONSUMER_KEY = 'HMnNDIlLnpxqoyAQC9gSs1ZxX'
CONSUMER_SECRET = 'uba7NCMbtGiLUOSs3cPp2TlzuveBZhUlCkBLLhQiHJFqx9zmvf'
ACCESS_TOKEN = '1153613267195039744-7ZDSA7vWYGJJnylob0or1tj6WHmoN0'
ACCESS_TOKEN_SECRET = 'xGpjqcAcEwzpCvG57DWghb4FwNfjV81UYuavTmxVZiVOb'
HOST = "127.0.0.1"  # Get local machine
PORT = 9999  # Reserve a port for your service.


In [2]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

import socket
import json
import logging
import threading

import twitter_credentials


In [3]:
LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
             -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
             -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

In [4]:
# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)
        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets
    
    
# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator:

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth


# # # # TWITTER STREAMER # # # #
class TwitterStreamer:
    """
    Class for streaming and processing live tweets.
    """

    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()

    def stream_tweets(self, hash_tag_list, c_socket):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(c_socket)
        auth = self.twitter_autenticator.authenticate_twitter_app()
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords:
        # stream.filter(track=hash_tag_list)
        stream.filter(locations=LOCATIONS, languages=["en"])


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """

    def __init__(self, c_socket):

        self.client_socket = c_socket

    def on_data(self, data):
        try:
            msg = json.loads(data)
            print(msg['text'].encode('utf-8'))
            self.client_socket.send(msg['text'].encode('utf-8'))
            self.client_socket.send("\n".encode('utf-8'))
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def on_error(self, status):
        print(status)
        return True


# create connection
def sendData(c_socket, hash_tag_list):
    twitter_streamer = TwitterStreamer()
    twitter_streamer.stream_tweets(hash_tag_list, c_socket)


def tread_function(port):
    s = socket.socket()  # Create a socket object
    host = twitter_credentials.HOST

    # Reserve a port for your service.
    s.bind((host, port))  # Bind to the port

    hash_tag_list = []
    #hash_tag_list = ["donal trump", "hillary clinton", "barack obama", "bernie sanders", "movie"]

    s.listen(100)  # Now wait for client connection.
    c, addr = s.accept()  # Establish connection with client.

    print("Received request from: " + str(addr))

    sendData(c, hash_tag_list)

In [5]:
x1 = threading.Thread(target=tread_function, args=(9999,))
x1.start()

Received request from: ('127.0.0.1', 59451)
b'Just posted a photo @ Oakland, California https://t.co/oPCVIKRhgt'
b'i peep the choosy shit, keep it away from me'
b"@TheReal_Hurl Here's one Bill Maher invented on Friday:\nYou make my heart race like COVID through a New York state nursing home."
b'@triplethefield @CheapFunBreaks \xf0\x9f\x98\x82 Kind of weird but funny'
b'Wow I\xe2\x80\x99ve been on Twitter for 12 years!\n\n*checks 12 years worth of tweets to make sure I didn\xe2\x80\x99t post something that\xe2\x80\xa6 https://t.co/7biCpkEcpZ'
b'@RSchwartz @govertime nobody does i don\xe2\x80\x99t think. i didn\xe2\x80\x99t even care if we lost at that point. i was proud johansen stood up for the captain'
b'@zanubaaaaa what kind of toys \xf0\x9f\xa4\xa8'
b'It\xe2\x80\x99s so good.'
b'Nice toes bruh \xf0\x9f\x98\x8d https://t.co/tDqAZej3ac'
b'I miss playing flappy bird'
b'Labreaboyy'
b"@beatlejuice3333 @FudFitt @iveyrenee @TeriyakiBeef4 @cnnbrk Here's where you dipshits get it wrong. No o

b'Whose up'
b'@danwootton That has been their brand for awhile now.'
b'Finally someone who not a hater said it \xf0\x9f\xa5\xba\xf0\x9f\xa5\xb0\xe2\x9d\xa4\xef\xb8\x8f'
b'\xf0\x9f\x91\x81\n\xf0\x9f\x91\x81guide me\xf0\x9f\x91\x81 by @marcosalvarado_ https://t.co/jFWpg2f90c'
b'Open the pool! Haha! https://t.co/mCJN5AZWNO'
b'Saved $1920 last Year , unplugging The Boob Tube!'
b'I need some woods'
b"@BoobieMilesXXIV hoping you're having a good off-season, can't wait to see you tear it up next season! #Eagles\xe2\x80\xa6 https://t.co/5NTeTWZaHp"
b'I need a different job, idk how much longer I can wake up at this time'
b'Me and my best friend both \xf0\x9f\x8d\xac in different states \xf0\x9f\xa5\xba\xf0\x9f\x92\x95 @colorless29'
b'I love it.'
b'Don\xe2\x80\x99t let the innocent face fool you https://t.co/OexaobGR5D'
b'Houston Tommorow what we doing \xf0\x9f\x91\x80'
b'@RevengeBunny @PyrPaws @reddogsusie @WinglessBird_ @SandraK93322487 @for_pittys @Janetlynne211 @4paws8 @jijmpel\xe2\x80\xa6 

b"\xf0\x9f\xab\x91Here\xe2\x80\x99s a song for you\xe2\x80\xa6 My Best Friend's Girl by The Cars\nhttps://t.co/pighpQO2Is"
b'@SpCeTK im jusy never home lol but then again I dont even fuck wit my fam like that (blacksheep) lol they weird af \xf0\x9f\x96\x95\xf0\x9f\x8f\xbe'
b'Wer my chicki baby @'
b'@adventurejub JAYCE LMFAAAOOOO'
b'I\xe2\x80\x99m not tryna win your heart'
b'oh wide awake, wby?'
b'self-sabotage or maybe not.'
b'Getting out of work early again\xf0\x9f\x98\x8e if I want to...\xf0\x9f\xa4\x94'
b'@Chloe46868713 Oh yeah baby, I will love it to focken you really good until both come over mamasita,te la mamo bien rrico baby.\xf0\x9f\x98\x9b\xf0\x9f\x98\x9d\xf0\x9f\x98\x9c\xf0\x9f\xa4\xaa\xf0\x9f\x98\x9d\xf0\x9f\x98\x9b\xf0\x9f\x98\x8b\xf0\x9f\x98\x8b'
b'@_prettyblackkk I o u\xe2\x80\x99s lol send it we can play for fun'
b'Let\xe2\x80\x99s make it illegal to put tik toks on your story.'
b'@ABC I FIND IT very very tough or hard to BELIEVE that the SKIN COLOR of Archy was a TOPIC

b'@Ashh_Ahh_Leee Lol I just tried to look in the comments for you, shit we alll wann know \xf0\x9f\x98\x82\xf0\x9f\x98\x82cause a bite is odeee! \xf0\x9f\x98\x82\xf0\x9f\xa4\xa6\xf0\x9f\x8f\xbd\xe2\x80\x8d\xe2\x99\x82\xef\xb8\x8f'
b'Is it just me or that march is going by really fast like usain bolt \xe2\x9a\xa1\xef\xb8\x8f'
b'But ain\xe2\x80\x99t nothing important to her'
b'Today Shan State, PeKon: Ten of thousands of people are peacefully protesting against dictatorship.\xe2\x80\xa6 https://t.co/z0MWrfrFlw'
b'TEST_GEO_Hierarchy: 3b2b8748-2ec6-4912-b871-038b003fd14f'
b'TEST_GEO_Hierarchy: 42dc59b8-7014-4255-a199-94d7f0ef2dc6'
b'I\xe2\x80\x99m tired of fake love'
b'@aiiendaniel i\xe2\x80\x99m going to sleep have fun tho'
b'It\xe2\x80\x99s easy to watch... just click here \xf0\x9f\x91\x89\xf0\x9f\x8f\xbb https://t.co/ozybP23pdT \xf0\x9f\x93\xba'
b'@ntm91307 @leninisttraitor @rmindler @AOC Or everyone else can pay $15 and you can work your own counters or wait your own tables \xf0\x9f\x9

b'who up ? In SE DC #dmvfreaks'
b'AW FUCK. ME REALIZING 7 YEARS LATER MY TORTOISES WERENT DEAD AND JUST HIBERNATING \xf0\x9f\x92\x80'
b'IGHT that\xe2\x80\x99s it, i\xe2\x80\x99m \xf0\x9f\xa6\x86ing simping'
b"Aereo-Plain.\n\nI don't dream much... and I almost never remember them when I wake....\n\nBut sometimes when I wake up\xe2\x80\xa6 https://t.co/453IOmSQpN"
b'If you were really trying I don\xe2\x80\x99t think I\xe2\x80\x99d feel this way can\xe2\x80\x99t even talk to you about my shit but you got me?'
b"@Spooly5 @milenaac @danwootton We know what they did to diana. It seems y'all have forgot. \xf0\x9f\x98\x82"
b'This is what @Sen_JoeManchin\n said on cnn about minimum wage? https://t.co/qLkvABIZzX'
b'@JamesRed_1981 @rdrunner_ @CNN @CityofQuincy @AOC SWINE!!!!! https://t.co/xgKol2vNea'
b'@Ninavote1 I hope you enjoyed \xf0\x9f\xa5\xb0'
b'Luv yuh Dell \xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3\xf0\x9f\xa4\xa3\xf0\x9f\xa7\xa1\xf0\x9f\x92\xab'
b'@AllieGoertz LOL I totally thought

...

b'Truthfully'
b'Finally caught up on Thurs episode, wow finally Peter is exposed of all the crimes he done. Waited a long time 4 th\xe2\x80\xa6 https://t.co/EcaginLgCI'
b'Mental illness fr LMAOOOO'
b'Dimples are so cute bruv dkmsmkakka'
b'@IvanKatey yes, yes and no please \xf0\x9f\x98\x8d'
b'@dvel86 I have slept a grand total of 30 minutes....ugh'
b'Harry chose to spend 10 years of his life in an Apache helicopter in Afghanistan over being ANYWHERE NEAR the Royal\xe2\x80\xa6 https://t.co/S1WNvQbtXw'
b'@indiedynamo Prince Philip is straight up 706 territory though; that\xe2\x80\x99s zombie FX makeup.'
b'@megohime1204 Thank you \xe2\x9c\x8c'
b'Why I gotta go to work so early\xf0\x9f\x98\xad\xf0\x9f\x98\xa9'
b'don\xe2\x80\x99t sit on US dollars, US debt goes up as the crypto market grows. back yourself and be protected'
b'Late night things https://t.co/U2prnrqKdF'
b'I\xe2\x80\x99m just up'
b'@Heathernab Good luck, from 3700 miles away'
b'@Ashley83530499 @Chloe46868713 Oh yeah baby, I will

b'sooo like anyone wanna hold hands romantically https://t.co/fI8J1ZPHMh'
b'Turns out the cure is being their (step) dad for a bit until they sort out their shit.'
b'@GundyGirl @trump_enemy Republicans failed to get the support of the American people. #RepublicanIMPOTENCE #RepublicansAreTheProblem'
b'@MagicalOverload Stp racers.'
b"@_MadeRealRell Wish I was there but I'm in Columbus...."
b'@_Mav3rick_7 @KuddlesK @nba_topshot @NBA @BalesFootball @Wades_ETH @jacksettleman @TheFirstMint @HighUpsideShow\xe2\x80\xa6 https://t.co/9uSIGHvyEX'
b'@ana_couper I would recommend a rebranding to \xe2\x80\x9cArt is Science\xe2\x80\x9d, but yes\xe2\x80\x94 Onward!! \xf0\x9f\x9a\x80\xe2\x9c\xa8'
b'Just a reminder... Chinese stocks have led the US in many stock downturns. As @DanielTNiles knows...'
b'Our Current Administration is making necessary implementations\nPresident Joe Biden and VP\nKamala Harris are meeting\xe2\x80\xa6 https://t.co/rJY2K1E4wc'
b'Thank you for creating a film I\xe2\x80\x99ve se

b"@Clara40136151 @Chloe46868713 Well just the way you're, I will focken both pussy and your delisus ass Sweetie.\xf0\x9f\x98\x8a\xf0\x9f\x98\x81\xf0\x9f\x98\x8d\xf0\x9f\xa5\xb0\xf0\x9f\x98\x98\xf0\x9f\x98\x98\xf0\x9f\x98\xa2\xf0\x9f\x98\x98"
b"@RepDonBeyer let's have a hand for our lovely physician's assistant. @CDCDirector https://t.co/YkiWAK3rYf"
b'i need a hug ):'
b"@Oilfield_Rando He'd be getting less flack if he divorced her and broke out that Hitler uniform we was vilified for\xe2\x80\xa6 https://t.co/eTgRDol7QX"
b'this breaks my heart \xf0\x9f\xa5\xba'
b'@mouthymouthy @sav_britt @DaniLeigh Run them DNA results. Because A) unless you come from African decent you can ST\xe2\x80\xa6 https://t.co/hFb9WNPMiK'
b'\xe2\x9c\x8c\xf0\x9f\x8f\xbeHouston, TX  until the next time.'
b'@hiphopsince94 You said Men it\xe2\x80\x99s a general statement lmao and furthermore I think we sympathize with it now more than it ever has been.'
b'Last night encouragement talks from yours truly to my sister \

b'Am I the only person that thinks @songbirdnsh_t Twitter name is this? https://t.co/EOESO5TtrF'
b'People who give a shit about the generations are so annoying.'
b'idk why lately i\xe2\x80\x99ve been having sm anxiety at night, it\xe2\x80\x99s like my body feels so many vibrations all over which make\xe2\x80\xa6 https://t.co/XSy84CCBtB'
b'All I ever wanted to do was help cure people\xe2\x80\x99s trauma/PTSD/whatever.\n\nIt\xe2\x80\x99s not MY fault(?) so many lovely ladies ha\xe2\x80\xa6 https://t.co/Xk5XsUlqWo'
b'@wsupcaramel I ain\xe2\x80\x99t mad at ya lol \xf0\x9f\x98\x82'
b'Truly relatable... https://t.co/TQqPOcwvsO'
b'@DannySenseDJ Oh good at least at $0 my portfolio can stop dropping LOL'
b'Accident in #Orange on Sand Lk Rd/SR 482 Both EB/WB before Orange Ave #traffic https://t.co/YKhnnOgYFK'
b"Closed due to accident in #TriStateTollwayI294 on Tri-State (I-294) NB from N Ave to O'Hare Oasis #CHITraffic https://t.co/GCesNZoWap"
b'Accident in #Orange on John Young Pky NB before SR

b'replace the dunk contest w horse'
b"@gaymoomin It won't make them any less predatory."
b'@SAfmRadio \n\n Check on McKenna Chambers and Nairobi Chambers I am very proud of my daughters and bring them gifts a\xe2\x80\xa6 https://t.co/NQgeIUbAgS'
b'As a fan, I never really had to question @kylekuzma authenticity. I appreciate that in a world full of flakey people.'
b'Bro, the movie called #TheRental got me not wanting to stay in a #airbnb real talk! https://t.co/F5I5aIok5I'
b'@bearmccreary @stevendeknight But we must reach across the aisle and work wit- https://t.co/Oxq3IlSQRp'
b'bro i never been tire all day \xf0\x9f\x98\xad'
b'Baby girl gotta start giving herself more credit as a single parent of three. If your children are happy then you are doing just fine.'
b'Keep it coming'
b'My ACE \xe2\x9c\x8a\xf0\x9f\x8f\xbe\xf0\x9f\x99\x8f\xf0\x9f\x8f\xbe\xe2\x9d\xa4\xef\xb8\x8f\xf0\x9f\x8f\x80\xf0\x9f\x92\xaa\xf0\x9f\x8f\xbe https://t.co/T3LxrEM4u7'
b'Rocket Watts takes NBA type shots.'
b'TES